In [83]:
import pymysql
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
from flask import Flask, request, jsonify
import base64
from flask_cors import CORS
from io import BytesIO

# Flask 애플리케이션 생성
app = Flask(__name__)
CORS(app)

# 연결에 필요한 정보
rds_host = 'capstone-database.c5ys4ks8sbyz.us-west-2.rds.amazonaws.com'
rds_port = 3306
rds_user = 'admin'  
rds_password = 'capstone'  
rds_database = 'capstone'  

# MySQL 연결
conn = pymysql.connect(
    host=rds_host,
    port=rds_port,
    user=rds_user,
    password=rds_password,
    database=rds_database
)

In [85]:
def generate_graph(data):
    # Pandas DataFrame으로 변환
    df = pd.DataFrame(data, columns=["Time", "Pop"])

    # x 축과 y 축 데이터 추출
    times = pd.to_datetime(df["Time"])
    pops = df["Pop"]

    # 그래프 생성
    plt.figure(figsize=(15, 6))
    plt.plot(times, pops, marker='o', color='blue')

        # 그래프 설정
    plt.xlabel("Time")
    plt.ylabel("Pop")
    plt.title("Pop vs. Time")
    plt.grid(True)

        # x 축 레이블 설정
    plt.xticks(times, rotation=45)

        # y 축 범위 설정
    plt.ylim(ymin=0, ymax=1)

        # 레이아웃 조정
    plt.tight_layout()
    
    # 이미지 파일로 저장
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    
    # 이미지 파일을 base64로 인코딩하여 반환
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return encoded_image

In [86]:
@app.route('/drawtable', methods=['POST'])
def send_table_name():
    content = request.get_json()
    table_name = content["table_name"]
    
    # 샘플 쿼리 실행
    with conn.cursor() as cursor:
        # 쿼리 작성
        query = f"SELECT time, date, pop FROM {table_name}"
        
        # 쿼리 실행
        cursor.execute(query)
        
        # 결과 가져오기
        result = cursor.fetchall()
        
        # 데이터 처리
        formatted_data = []
        for item in result:
            # 시간과 날짜 추출
            time = item[0]  # timedelta
            date = item[1]  # date
            pop = item[2]
            
            # datetime 객체 생성
            datetime_obj = datetime.combine(date, datetime.min.time()) + time
            
            # 데이터 포맷 변경
            formatted_data.append((datetime_obj, pop))
            
        # 데이터 정렬
        formatted_data = sorted(formatted_data, key=lambda x: x[0])
        img = generate_graph(formatted_data)
        
        return jsonify({"message": "POST request received", "encoded_image": img})

In [87]:
@app.route('/mao')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
     app.run(host='0.0.0.0', port=8000)  # 포트는 원하는 포트로 변경 가능

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
